In [1]:
import getpass
import os
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders.csv_loader import CSVLoader
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",

)

In [3]:
df=pd.read_csv('data/product_reviews.csv')

In [4]:
loader = CSVLoader(file_path="data/product_reviews.csv",source_column='product_title',
                   csv_args={
                       'delimiter': ',',
                       "fieldnames": ['product_title','rating','summary','review'],
                   })

documents = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [7]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [8]:
vectorstore.save_local("faiss_index_openai")

In [9]:
results = vectorstore.similarity_search_with_score(
    "best performance product",
    k=2,
)

In [11]:
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.154299] product_title: OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset
rating: 5
summary: Simply awesome
review: Awesome looking and stylish all my friends are asking me for testing this product and some of them are buying. that's batter and I like that thing. Also the phone has low latency mode that's also help me in game overall good performance. Must buy 🤗 [{'source': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'row': 226}]
* [SIM=1.204979] product_title: OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset
rating: 5
summary: Super!
review: Amazing productSound quality super se upper 💓Battery backup A1 [{'source': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset', 'row': 240}]


In [1]:
## Loading persistent db

In [12]:
persisted_vectorstore = FAISS.load_local("faiss_index_openai", embeddings,allow_dangerous_deserialization=True)

In [13]:
def get_answer(question):
    answer=persisted_vectorstore.similarity_search_with_score(question, k=5)
    answer= [{(s.split(":")[0]):(s.split(":")[1]) for s in a[0].page_content.split("\n")} for a in answer]
    return [a for a in answer if len(a)>1]

    

In [14]:
q="Best earphones with great sound quality"

In [15]:
answer=(get_answer(q))

In [16]:
answer

[{'product_title': ' OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset',
  'rating': ' 5',
  'summary': ' Brilliant',
  'review': ' Best sound & bass quality'},
 {'product_title': ' BoAt BassHeads 100 Wired Headset',
  'rating': ' 5',
  'summary': ' Just wow!',
  'review': " Best budgeted earphones, build quality is so good that you can fold them without any fear of tearing. Best part is it's bass and an awesome sound experience. Best earphones till used by me. ❤"},
 {'product_title': ' OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset',
  'rating': ' 4',
  'summary': ' Worth the money',
  'review': ' Super sound quality'},
 {'product_title': ' realme Buds 2 Wired Headset',
  'rating': ' 5',
  'summary': ' Excellent',
  'review': ' nice earphones with smooth sound. even the build quality is best . this earphone is better for bass lover . and also  for smooth playback'},
 {'product_title': ' OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset',
  'rating': ' 5',
  

In [31]:
answer=persisted_vectorstore.similarity_search_with_score("Best earphones with great sound quality", k=2)


In [32]:
print((answer)[0][0].page_content)

rating: 5
summary: Just wow!
review: Best quality earphone ever I used excellent sound quality superb battery backup looking Also Good fully worth
